# Program to solve heat diffusion equation in a rectangular domain with imposed flux on L_left

In [1]:
import numpy as np
from mpi4py import MPI
import pyvista
import ufl

from dolfinx import fem, plot
from dolfinx.io import XDMFFile
from dolfinx.fem import FunctionSpace, Constant

from petsc4py.PETSc import ScalarType

## Read the mesh

In [2]:
# Read the mesh
with XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cell_tags = xdmf.read_meshtags(mesh, name="Grid")
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)

with XDMFFile(MPI.COMM_WORLD, "facet_mesh.xdmf", "r") as xdmf:
    facet_tags = xdmf.read_meshtags(mesh, name="Grid")

## Plot the mesh

## Define temporal parameters

In [3]:
t = 0 # Start time
T = 2.0 # Final time
num_steps = 61     
dt = T / num_steps # time step size

## Finite element function space

In [4]:
V = FunctionSpace(mesh, ("CG", 1))  # Lagrange element and and linear elements (degree 1)

## Set initial conditions

In [5]:
u_n = Constant(mesh, ScalarType(0))

## Boundary conditions

In [6]:
# DIRICHLET: p=0 on all Dirichlet BC (sides 1, 2 and 4)
boundary_dofs = fem.locate_dofs_topological(V, mesh.topology.dim-1, facet_tags.indices[facet_tags.values != 3])
bc = fem.dirichletbc(ScalarType(0), boundary_dofs, V)

# NEUMANN: dp/dt=-10 on the left side (side 3)
x = ufl.SpatialCoordinate(mesh)
g = -10 * x[1]

## Trial and test functions

In [7]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)

## Source term

In [8]:
# here f=0 as there is no source term
f = fem.Constant(mesh, ScalarType(0))

## Variational problem

As the test function is zero on the boundary integrals over the Dirichlet boundary, we can integrate g*v*ds over the entire boundary.

In [9]:
#a = u * v * ufl.dx + dt*ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 
#L = (u_n + dt * f) * v * ufl.dx - dt * g * v * ufl.ds

a = u * v * ufl.dx + dt*ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 
L = (u_n + dt * f) * v * ufl.dx

## Preparation of linear algebra structures for time dependent problems

In [10]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

ArityMismatch: Failure to conjugate test function in complex Form

## Compute the error

In [ ]:
# first get the analytical solution at each node 
V2 = fem.FunctionSpace(mesh, ("CG", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)
# compute L2 error 
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(mesh.comm.allreduce(error_local, op=MPI.SUM))
print(f"Error_L2 : {error_L2:.2e}")

## Plot the solution uh

In [ ]:
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
u_plotter.show()